In [1]:
import lsdb
from tape import Ensemble, ColumnMapper
import matplotlib.pyplot as plt
import dask
dask.config.set({'temporary_directory': '/data/epyc/users/brantd/tmp'})

from dask.distributed import Client
client = Client(n_workers=10, threads_per_worker=10,
                memory_limit="60G")

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 558.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33658,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 558.79 GiB
Comm: tcp://127.0.0.1:45180,Total threads: 10
Dashboard: http://127.0.0.1:33583/status,Memory: 55.88 GiB
Nanny: tcp://127.0.0.1:40061,


# Generating a custom-sized subset of ZTF for TAPE Analysis

In [2]:
# path to ZTF AXS
ztf_object_path = "/astro/store/epyc3/data3/hipscat/catalogs/ztf_axs/ztf_dr14"
ztf_source_path = "/astro/store/epyc3/data3/hipscat/catalogs/ztf_axs/ztf_source/"

In [3]:
#Load and Subsample using LSDB

object_cols = ["ps1_objid", "ra", "dec"]
source_cols = ["ps1_objid", "ra", "dec", "mjd", "mag", "magerr", "band"]

# Load into LSDB catalog objects
ztf_object = lsdb.read_hipscat(ztf_object_path) # ZTF Object
ztf_source = lsdb.read_hipscat(ztf_source_path, columns=source_cols) # ZTF Source

# Box Search to filter down to a small subset
ra = (340, 342)
dec = (10, 12)
ztf_object = ztf_object.box(ra=ra, dec=dec)
ztf_source = ztf_source.box(ra=ra, dec=dec)

# Join Source to Object to set proper object-level _hipscat_index
joined_source = ztf_object.join(
    ztf_source, left_on="ps1_objid", right_on="ps1_objid", suffixes=("_object", "")
)

In [4]:
# Load into TAPE

# ColumnMapper Establishes which table columns map to timeseries quantities
colmap = ColumnMapper(
        id_col='_hipscat_index',
        time_col='mjd',
        flux_col='mag',
        err_col='magerr',
        band_col='band',
      )

ens = Ensemble(client=client)

ens.from_lsdb(joined_source, ztf_object, column_mapper=colmap)

In [5]:
print(ens.object.npartitions, ens.source.npartitions)

3 11


In [9]:
len(ens.object)

75944

In [6]:
# Save to parquet for easy loading
import numpy as np
ens.source.index=ens.source.index.astype(np.uint64) # need this until lsdb 0.2
ens.save_ensemble(".", "ztf_small_ensemble")

/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 42.48 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Saved to ./ztf_small_ensemble
